# Meent Tutorial 2
## PyTorch: Gradient and Optimization

In [1]:
import torch

import meent
from meent.on_torch.optimizer.loss import LossDeflector
from meent.on_torch.optimizer.optimizer import OptimizerTorch

In [2]:
backend = 2  # Torch

# common
pol = 0  # 0: TE, 1: TM

n_I = 1  # n_incidence
n_II = 1  # n_transmission

theta = 20 * torch.pi / 180
phi = 50 * torch.pi / 180

wavelength = 900

thickness = torch.tensor([500.])
period = torch.tensor([1000., 1000.])

fourier_order = [4, 2]

type_complex = torch.complex128
device = torch.device('cpu')

grating_type = 2

In [3]:
ucell_1d_s = torch.tensor([
    [
        [0, 1, 0, 1, 1.1, 0, 1, 0, 1, 1, ],
    ],
], dtype=torch.float64) * 4. + 1.  # refractive index

ucell_2d_s = torch.tensor([
    [
        [0, 1, 0, 1, 1, 0, 1, 0, 1, 1, ],
        [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ],
    ],
]) * 4 + 1.  # refractive index

## 2.1 Gradient

In [4]:
mee = meent.call_mee(backend=backend, grating_type=grating_type, pol=pol, n_I=n_I, n_II=n_II, theta=theta, phi=phi, fourier_order=fourier_order, wavelength=wavelength, period=period, ucell=ucell_2d_s, thickness=thickness, type_complex=type_complex, device=device, fft_type=0, improve_dft=True)

In [5]:
pois = ['ucell', 'thickness']

forward = mee.conv_solve
loss_fn = LossDeflector(x_order=0, y_order=0)

In [6]:
# case 1: Gradient
grad = mee.grad(pois, forward, loss_fn)
print(1, grad)

1 {'ucell': tensor([[[-0.2078,  0.0844,  0.2179, -0.1286, -0.0780, -0.1279,  0.3122,
           0.0245, -0.1547, -0.1577],
         [-0.0771,  0.0381,  0.0450, -0.1212, -0.0914,  0.1777,  0.0569,
          -0.0266, -0.0197, -0.0962]]]), 'thickness': tensor([-0.0053])}


In [7]:
# case 2: SGD
opt_torch = torch.optim.SGD
opt_options = {'lr': 1E-2,
               'momentum': 0.9,
               }

mee.fit(pois, forward, loss_fn, opt_torch, opt_options)

step 0, loss: 0.15493761782593868
step 1, loss: 0.15111075431034662
step 2, loss: 0.14183186997260694
step 3, loss: 0.12332711779095337
step 4, loss: 0.08856972837925749
step 5, loss: 0.07092493684710922
step 6, loss: 0.14600607702747972
step 7, loss: 0.0869572184627041
step 8, loss: 0.08144524682914499
step 9, loss: 0.14457611471291892
step 10, loss: 0.16933163648063215
step 11, loss: 0.15881475429847008
step 12, loss: 0.059979157460718927
step 13, loss: 0.0897823464124526
step 14, loss: 0.07209184286785993
step 15, loss: 0.10810032905842401
step 16, loss: 0.16805218345285086
step 17, loss: 0.13112988778394638
step 18, loss: 0.07915957071252011
step 19, loss: 0.1901704162249384
step 20, loss: 0.1354443989009719
step 21, loss: 0.10122597692998171
step 22, loss: 0.10255479573012426
step 23, loss: 0.12356671105317445
step 24, loss: 0.11528691895712162
step 25, loss: 0.15758294186009317
step 26, loss: 0.13147680376676926
step 27, loss: 0.16683191402863354
step 28, loss: 0.1352199993786537